In [1]:
from utils.mt_5.data import get_currency_pair_data_
symbol = "GBPJPY"
df = get_currency_pair_data_(symbol, timeframe='m15', years_back=.5)
df


,close
time,
2024-03-22 23:45:00,190.777
2024-03-25 00:00:00,190.660
2024-03-25 00:15:00,190.717
2024-03-25 00:30:00,190.659
2024-03-25 00:45:00,190.712
...,...
2024-12-04 16:15:00,191.309
2024-12-04 16:30:00,191.228
2024-12-04 16:45:00,191.405


In [2]:

from utils.ts import make_lags, make_multistep_targets

X = make_lags(ts=df, lags=10).fillna(0.0)
y = make_multistep_targets(ts=df, steps=10).dropna()

# Align X and y shapes by dropping the first rows from X
X = X.iloc[9:].reset_index(drop=True)  
y = y.reset_index(drop=True) 

In [3]:
X.iloc[-1]

y_lag_1   close    191.330
y_lag_2   close    191.405
y_lag_3   close    191.228
y_lag_4   close    191.309
y_lag_5   close    191.461
y_lag_6   close    191.457
y_lag_7   close    191.313
y_lag_8   close    191.419
y_lag_9   close    191.319
y_lag_10  close    191.311
Name: 17522, dtype: float64

In [4]:
y.iloc[-1]

y_step_1   close    191.319
y_step_2   close    191.419
y_step_3   close    191.313
y_step_4   close    191.457
y_step_5   close    191.461
y_step_6   close    191.309
y_step_7   close    191.228
y_step_8   close    191.405
y_step_9   close    191.330
y_step_10  close    191.373
Name: 17522, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train

,y_lag_1,y_lag_2,y_lag_3,y_lag_4,y_lag_5,y_lag_6,y_lag_7,y_lag_8,y_lag_9,y_lag_10
,close,close,close,close,close,close,close,close,close,close
0,190.552,190.605,190.518,190.543,190.712,190.659,190.717,190.660,190.777,0.000
1,190.341,190.552,190.605,190.518,190.543,190.712,190.659,190.717,190.660,190.777
2,190.508,190.341,190.552,190.605,190.518,190.543,190.712,190.659,190.717,190.660
3,190.550,190.508,190.341,190.552,190.605,190.518,190.543,190.712,190.659,190.717
4,190.619,190.550,190.508,190.341,190.552,190.605,190.518,190.543,190.712,190.659
...,...,...,...,...,...,...,...,...,...,...
14013,195.499,195.518,195.490,195.538,195.523,195.583,195.539,195.546,195.484,195.534
14014,195.554,195.499,195.518,195.490,195.538,195.523,195.583,195.539,195.546,195.484
14015,195.673,195.554,195.499,195.518,195.490,195.538,195.523,195.583,195.539,195.546


In [6]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

model = MultiOutputRegressor(XGBRegressor())
model.fit(X_train, y_train)

last_lags = X.iloc[-1].values.reshape(1, -1)
y_fore = model.predict(last_lags) 

y_fore

array([[191.30873, 191.38905, 191.3047 , 191.45673, 191.45833, 191.30533,
        191.22537, 191.39232, 191.34528, 191.31703]], dtype=float32)

In [7]:
import pandas as pd

future_timestamps = pd.date_range(df.index[-1] + pd.Timedelta(minutes=15), periods=10, freq='15min')

forecast_df = pd.DataFrame({
    "time": future_timestamps,
    "predicted_close": y_fore.flatten()
})

forecast_df.index = forecast_df['time']
forecast_df.drop('time', axis=1, inplace=True)

forecast_df

,predicted_close
time,
2024-12-04 17:30:00,191.308731
2024-12-04 17:45:00,191.389053
2024-12-04 18:00:00,191.304703
2024-12-04 18:15:00,191.456726
2024-12-04 18:30:00,191.458328
2024-12-04 18:45:00,191.305328
2024-12-04 19:00:00,191.225372
2024-12-04 19:15:00,191.392319
2024-12-04 19:30:00,191.345276


In [8]:
current_price = df['close'].iloc[-1] 
predicted_close = round(forecast_df['predicted_close'].iloc[-1], 3)

if predicted_close > current_price: # buy
    print("predicted_close is GREATER than current_close")
    target_profit = round((predicted_close - current_price) * 100)

if predicted_close < current_price: # short-sell
    print("We will close LOWER!")
    target_profit = round((current_price - predicted_close) * 100)
    

print(current_price, predicted_close, target_profit)


We will close LOWER!
191.373 191.317 6


In [10]:
from utils.mt_5.trade import buy, short_sell, sell

current_price = df['close'].iloc[-1] 
predicted_day_close = round(forecast_df['predicted_close'].iloc[-1], 5)  

past_1800hrs = df.index.time[-1] >= pd.Timestamp('20:00:00').time()

if not past_1800hrs:
    if predicted_day_close > current_price: # buy
        print("predicted_day_close is GREATER than current_close\n\n....BUYING....\n")
        target_profit = round((predicted_day_close - current_price) * 100)
        buy(symbol, target_profit=target_profit)

    elif predicted_day_close < current_price: # short-sell
        print("We will close LOWER!\n\n.....SHORT-SELLING.....\n")
        target_profit = round((current_price - predicted_day_close) * 100)
        short_sell(symbol, target_profit=target_profit)
    else:
        print("....I DON'T KNOW HOW TO HELP YOU!....")
else: # sell
    print("You CANNOT trade PAST 1800hrs\n\n....SELLING....\n")
    sell(symbol)


We will close LOWER!

.....SHORT-SELLING.....

GBPJPYm already short-sold!
